In [12]:
from openai import OpenAI
from dotenv import load_dotenv
import json

load_dotenv()
client = OpenAI()

# Create Clarifying Agent

In [9]:
mark_completed_object = {
    "type": "function",
    "function": {
        "name": "mark_completed",
        "description": "Marks the overall clarifying task as completed. Ends the session with the user, and passes the task to the Planner Agent",
    }
}

prompt_text_object = {
    "type": "function", 
    "function": {
        "name": "prompt_text",
        "description": "Prompt the user for plain text response",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "The text to display to the user to prompt for input"
                }
            },
            "required": ["text"]
        }
    }
}

prompt_list_object = {
    "type": "function",
    "function": {
        "name": "prompt_list",
        "description": "Give the user a list of options to choose from",
        "parameters": {
            "type": "object",
            "properties": {
                "options": {
                    "type": "array",
                    "description": "The list of options to display to the user to choose from",
                    "items": {
                        "type": "string",
                        "description": "One option"
                    }
                },
                "select_amount": {
                    "type": "string",
                    "enum": ["single", "multiple"]
                },
                "text": {
                    "type": "string",
                    "description": "The text to display to the user to prompt for input"
                }
            },
            "required": ["options", "select_amount", "text"]
        }
    }
}



clarifying_agent = client.beta.assistants.create(
    name="Clarifying Agent",
    model="gpt-4-1106-preview",
    instructions="""You are an assistant created by Apple to help users achieve tasks on their Macbooks automatically. Specifically, your role is to clarify the user's requirements by asking important questions. After clarification, the user's answers and their overall goal will be passed to the Planner Agent to create steps.
        
    Example:
    User: Text "happy birthday" to Peter Liu
    Assistant: prompt_text(text="What platform would you like to send the text on?")
    User: iMessage
    Assistant: prompt_list(options=["Auto-find contact from contacts list", "Specify phone number to send to"], select_amount="single", text="How should I find Peter's information?")
    User: Scrape from contacts
    Assistant: mark_completed()
    """,
    tools=[mark_completed_object, prompt_text_object, prompt_list_object],
)

In [21]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Send a text to everyone whose birthday is today"
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=clarifying_agent.id
)

In [22]:
import time

def wait_for_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        time.sleep(0.5)
    print("Run is now " + run.status)
    return run

In [23]:
completed = False
while not completed:
    run = wait_for_run(run, thread)
    while run.status in ("completed", "failed", "expired", "cancelled"):
        print('Restarting run')
        client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content="I can't read this output! If you're trying to mark the task as completed, please call `mark_completed`. And if you're trying to talk to me, please call a prompting method"
        )
        
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=clarifying_agent.id
        )
        
        run = wait_for_run(run, thread)
    
    calls = run.required_action.submit_tool_outputs.tool_calls
    tool_outputs = []
    for tool_call in calls:
        obj = {
            "tool_call_id": tool_call.id,
            "output": ""
        }
        
        func = tool_call.function
        arguments = json.loads(func.arguments)
        name = func.name
        
        print(name)
        
        if name == "mark_completed":
            completed = True
            break
        elif name == "prompt_text":
            obj["output"] = input(arguments["text"])
        elif name == "prompt_list":
            obj["output"] = input(arguments["text"] + "\n\n" + "\n".join(arguments["options"]))
        
        obj["output"] = json.dumps(obj["output"])
        tool_outputs.append(obj)
        
    if completed:
        break    
    
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs = tool_outputs
    )

Run is now requires_action
prompt_list
Run is now requires_action
prompt_text
Run is now requires_action
prompt_list
Run is now requires_action
mark_completed


BadRequestError: Error code: 400 - {'error': {'message': "Expected tool outputs for call_ids ['call_1e91Qwr0sVJw7b91hz3CAFr0'], got []", 'type': 'invalid_request_error', 'param': None, 'code': None}}